# 1. Import the required libraries

In [9]:
# Standard code libraries
import os
import platform
import glob

import numpy as np
from numpy.matlib import repmat
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt

# Custom code libraries from ReSurfEMG
from resurfemg.config.config import Config
from resurfemg.data_connector import file_discovery
from resurfemg.pipelines import ipy_widgets
from resurfemg.data_connector.tmsisdk_lite import Poly5Reader

%matplotlib widget

## 2. Load the ventilator and sEMG data

In [ ]:
# Identify all recordings available for the selected patient/measurement_date

# First find the patients
config = Config()
base_path = config.get_directory('root_patient_data_directory')
patient_paths = file_discovery.find_folders(
    base_path,
    folder_levels=['patient'])
patient = list(patient_paths['patient'].values)[0]

# Then find the files for the selected patients:
folder_levels = ['date', 'measurement']
patient_path = os.path.join(base_path, patient)

emg_files = file_discovery.find_files(
    base_path=patient_path,
    file_name_regex='EMG_recording',
    extension_regex='poly5',
    folder_levels=folder_levels)

folder_levels = ['date', 'measurement']
vent_files = file_discovery.find_files(
    base_path=patient_path,
    file_name_regex='Draeger_recording',
    extension_regex='poly5',
    folder_levels=folder_levels)

button_list = ipy_widgets.file_select(
    emg_files,
    folder_levels=folder_levels[:-1],
    default_value_select=None,
    default_idx_select=[1, 6])

In [ ]:
emg_file_chosen = os.path.join(
    patient_path, *[btn.value for btn in button_list], 'EMG_recording.Poly5')
vent_file_chosen = os.path.join(
    patient_path, *[btn.value for btn in button_list], 'Draeger_recording.Poly5')

print("The chosen files are:\n", emg_file_chosen, '\n', vent_file_chosen)

In [ ]:
# Load the EMG and ventilator data recordings from the selected folders.
data_emg = Poly5Reader(emg_file_chosen)
data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
fs_emg = data_emg.sample_rate
data_vent_samples = data_vent.samples[:data_vent.num_samples]
fs_vent = data_vent.sample_rate

# Define the time series of the EMG and ventilator recordings
y_emg = data_emg_samples[:, 1*fs_emg:61*fs_emg]
y_vent = data_vent_samples[:, 1*fs_vent:61*fs_vent]

# Define the time axes
t_emg = [i/fs_emg for i in range(len(y_emg[0, :]))]
t_vent = [i/fs_vent for i in range(len(y_vent[0, :]))]

In [ ]:
from resurfemg.visualization.visualization import show_my_power_spectrum, show_psd_welch
fig, axis = plt.subplots(figsize=(6, 6), sharex=True)
# show_my_power_spectrum(
#     y_emg[1, :], fs_emg, fs_emg//2, axis_spec=1,
# )

f, psd = show_psd_welch(y_emg[1, :], fs_emg, fs_emg, axis_spec=1, signal_unit='uV')

psd_smooth = pd.Series(psd).rolling(
        window=50,
        min_periods=1,
        center=True).mean()
axis.plot(f, psd_smooth)

In [ ]:
# Identify all recordings available for the selected patient/measurement_date

# Root directory for test data
config = Config()
root_patient_data_directory = \
    config.get_directory('test_data')

if platform.system() == 'Windows':
    path_sep = "\\"
else:
    path_sep = '/'

emg_pattern = os.path.join(root_patient_data_directory, '**/*.Poly5')
emg_and_vent_files = glob.glob(emg_pattern, recursive=True)

emg_files = []
vent_files = []

for file in emg_and_vent_files:
    if 'vent' in file:
        vent_files.append(file)
    else:
        emg_files.append(file)

emg_file_chosen_dummy = emg_files[0]
vent_file_chosen_dummy = vent_files[0]
print("The chosen files are:\n", emg_file_chosen_dummy, '\n', vent_file_chosen_dummy)

In [15]:
# Identify all recordings available for the selected patient/measurement_date

# First find the patients
config = Config()

# Then find the files for the selected patients:
base_path = config.get_directory('test_data')
folder_levels = None

emg_files = file_discovery.find_files(
    base_path=base_path,
    file_name_regex='emg_data_synth_pocc',
    extension_regex='poly5',
    folder_levels=None)

folder_levels = ['date', 'measurement']
vent_files = file_discovery.find_files(
    base_path=base_path,
    file_name_regex='vent_data_synth_pocc',
    extension_regex='poly5',
    folder_levels=None)

# button_list = ipy_widgets.file_select(
#     emg_files,
#     folder_levels=['files'],
#     default_value_select=None,
#     default_idx_select=[0])

In [ ]:
# Load the EMG and ventilator data recordings from the selected folders.
emg_file_chosen_dummy = os.path.join(base_path, emg_files['files'].values[0])
vent_file_chosen_dummy = os.path.join(base_path, vent_files['files'].values[0])


data_emg_dummy = Poly5Reader(emg_file_chosen_dummy)
data_vent_dummy = Poly5Reader(vent_file_chosen_dummy)
data_emg_dummy_samples = data_emg_dummy.samples[:data_emg.num_samples]
fs_emg_dummy = data_emg_dummy.sample_rate
data_vent_dummy_samples = data_vent.samples[:data_vent.num_samples]
fs_vent_dummy = data_vent.sample_rate

# Define the time series of the EMG and ventilator recordings
y_emg_dummy = data_emg_dummy_samples[:, 1*fs_emg:61*fs_emg]
y_vent_dummy = data_vent_dummy_samples[:, 1*fs_vent:61*fs_vent]

# Define the time axes
t_emg_dummy = [i/fs_emg_dummy for i in range(len(y_emg_dummy[0, :]))]
t_vent_dummy = [i/fs_vent_dummy for i in range(len(y_vent_dummy[0, :]))]

In [ ]:
from resurfemg.visualization.visualization import show_my_power_spectrum, show_psd_welch
fig, axis = plt.subplots(figsize=(6, 6), sharex=True)
# show_my_power_spectrum(
#     y_emg[1, :], fs_emg, fs_emg//2, axis_spec=1,
# )

f_dummy, psd_dummy = show_psd_welch(y_emg_dummy[1, :], fs_emg_dummy, fs_emg_dummy, axis_spec=1, signal_unit='uV')

psd_dummy_smooth = pd.Series(psd_dummy).rolling(
        window=50,
        min_periods=1,
        center=True).mean()
axis.plot(f_dummy, psd_dummy_smooth)

In [ ]:
fig, axis = plt.subplots(figsize=(6, 6), sharex=True)
axis.semilogy(f, psd_smooth/psd_dummy_smooth)

b, a = signal.butter(1, 100, 'low', analog=True)
b *= 2
w, h = signal.freqs(b, a, worN=[f_i * 2 * np.pi for f_i in f])
f_butter = w/(2*np.pi)
axis.semilogy(f_butter, np.abs(h), color='tab:green')
